# Sorting Scenes
 Used after data has been processed to aid in curating a dataset


#### Dependecy Import

In [ ]:
import numpy as np
import os
import scipy.interpolate
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from tkinter import Tk, Frame, StringVar, Label, Button, Radiobutton, BOTH, ttk
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from mpl_toolkits.basemap import Basemap
from matplotlib.patches import Polygon
import scipy.ndimage

#### Functions

In [ ]:
data_type = [
            ('xch4_corrected', 'f4', ()),
            ('latitude_corners', 'f4', (4,)),
            ('longitude_corners', 'f4', (4,)),
            ('u10', 'f4', ()),
            ('v10', 'f4', ()),
            ('latitude_center', 'f4', ()),
            ('longitude_center', 'f4', ()),
            ('scanline', 'i4', ()),
            ('ground_pixel', 'i4', ()),
            ('time', 'i4', (7,)),
            ('solar_zenith_angle', 'f4', ()),
            ('viewing_zenith_angle', 'f4', ()),
            ('relative_azimuth_angle', 'f4', ()),
            ('altitude_levels', 'f4', (13,)),
            ('surface_altitude', 'f4', ()),
            ('surface_altitude_stdv', 'f4', ()),
            ('dp', 'f4', ()),
            ('surface_pressure', 'f4', ()),
            ('dry_air_subcolumns', 'f4', (12,)),
            ('fluorescence_apriori', 'f4', ()),
            ('cloud_fraction', 'f4', (4,)),
            ('weak_h2o_column', 'f4', ()),
            ('strong_h2o_column', 'f4', ()),
            ('weak_ch4_column', 'f4', ()),
            ('strong_ch4_column', 'f4', ()),
            ('cirrus_reflectance', 'f4', ()),
            ('stdv_h2o_ratio', 'f4', ()),
            ('stdv_ch4_ratio', 'f4', ()),
            ('xch4', 'f4', ()),
            ('xch4_precision', 'f4', ()),
            ('xch4_column_averaging_kernel', 'f4', (12,)),
            ('ch4_profile_apriori', 'f4', (12,)),
            ('xch4_apriori', 'f4', ()),
            ('fluorescence', 'f4', ()),
            ('co_column', 'f4', ()),
            ('co_column_precision', 'f4', ()),
            ('h2o_column', 'f4', ()),
            ('h2o_column_precision', 'f4', ()),
            ('spectral_shift', 'f4', (2,)),
            ('aerosol_size', 'f4', ()),
            ('aerosol_size_precision', 'f4', ()),
            ('aerosol_column', 'f4', ()),
            ('aerosol_column_precision', 'f4', ()),
            ('aerosol_altitude', 'f4', ()),
            ('aerosol_altitude_precision', 'f4', ()),
            ('aerosol_optical_thickness', 'f4', (2,)),
            ('surface_albedo', 'f4', (2,)),
            ('surface_albedo_precision', 'f4', (2,)),
            ('reflectance_max', 'f4', (2,)),
            ('convergence', 'i4', ()),
            ('iterations', 'i4', ()),
            ('chi_squared', 'f4', ()),
            ('chi_squared_band', 'f4', (2,)),
            ('number_of_spectral_points_in_retrieval', 'i4', (2,)),
            ('degrees_of_freedom', 'f4', ()),
            ('degrees_of_freedom_ch4', 'f4', ()),
            ('degrees_of_freedom_aerosol', 'f4', ()),
            ('signal_to_noise_ratio', 'f4', (2,)),
            ('rms', 'f4', ())
        ]

channel_map = {}
current_channel = 0

for name, field_type, *field_shape in data_type:
    # Ensure current_channel is an integer
    current_channel = int(current_channel)

    if not field_shape:  # Scalar field
        channel_map[name] = slice(current_channel, current_channel + 1)
        current_channel += 1
    else:  # Multi-dimensional field
        total_channels = int(np.prod(field_shape))
        channel_map[name] = slice(current_channel, current_channel + total_channels)
        current_channel += total_channels

# Add the normalized methane variable as the last channel
channel_map['normalized_ch4'] = slice(current_channel, current_channel + 1)



In [ ]:
def calculate_new_bbox(min_lon, max_lon, min_lat, max_lat, padding=0):
    min_lon = np.round(min_lon, 1)
    min_lat = np.round(min_lat, 1)
    max_lon = np.round(max_lon, 1)
    max_lat = np.round(max_lat, 1)
    
    # Calculate the width and height
    width = max_lon - min_lon
    height = max_lat - min_lat

    # Find the maximum of width and height to maintain an equal aspect ratio
    max_dim = max(width, height)

    # Calculate the center of the bounding box
    center_lon = (min_lon + max_lon) / 2
    center_lat = (min_lat + max_lat) / 2

    # Calculate half of the max dimension
    half_dim = max_dim / 2

    # Calculate the new bounding box
    new_min_lon = center_lon - half_dim - padding
    new_max_lon = center_lon + half_dim + padding
    new_min_lat = center_lat - half_dim - padding
    new_max_lat = center_lat + half_dim + padding

    return [new_min_lon, new_max_lon, new_min_lat, new_max_lat]

#### Scene View Gui

In [ ]:
class SceneViewerApp:
    def __init__(self, root, years, locations, channel_map):
        self.root = root
        self.years = years
        self.locations = locations
        self.channel_map = channel_map
        self.scenes = []
        self.current_scene_index = 0  # Start with the first scene
        self.labels = []  # List to store labels
        self.plume_scenes = []  # List to store only plume scenes
        self.plume_count = 0  # Counter for the number of plumes

        # Tkinter variables for year and location selection
        self.selected_year = StringVar()
        self.selected_year.set(self.years[0])  # Default to the first year

        self.selected_location = StringVar()
        self.selected_location.set(self.locations[0])  # Default to the first location

        # Set up the Tkinter interface
        self.root.title("Scene Viewer")

        # Frame for year and location selection
        self.selection_frame = Frame(self.root)
        self.selection_frame.pack(side="top", fill=BOTH, padx=10, pady=10)
        
        # Year selection combo box
        self.year_combobox = ttk.Combobox(self.selection_frame, textvariable=self.selected_year, values=self.years)
        self.year_combobox.pack(side="left", padx=5)



        # Location selection combo box
        self.location_combobox = ttk.Combobox(self.selection_frame, textvariable=self.selected_location, values=self.locations)
        self.location_combobox.pack(side="left", padx=5)
        
        #self.channel_view = ttk.Combobox(self.selection_frame, textvariable=self.channel_map, values=list(channel_map.keys()))
        #self.channel_view.pack(side="left", padx=5)

        # Load scenes button
        self.load_button = Button(self.selection_frame, text="Load Scenes", command=self.load_scenes)
        self.load_button.pack(side="left", padx=5)

        # Frame for navigation controls
        self.control_frame = Frame(self.root)
        self.control_frame.pack(side="top", fill=BOTH, padx=10, pady=10)

        # Previous button
        self.prev_button = Button(self.control_frame, text="Previous", command=self.show_previous_scene)
        self.prev_button.pack(side="left", padx=5)

        # Label to show the current scene index
        self.scene_label = Label(self.control_frame, text="")
        self.scene_label.pack(side="left", padx=5)

        # Next button
        self.next_button = Button(self.control_frame, text="Next", command=self.show_next_scene)
        self.next_button.pack(side="left", padx=5)

        # Plume and No Plume buttons
        self.plume_button = Button(self.control_frame, text="Plume", command=lambda: self.label_scene(1))
        self.plume_button.pack(side="left", padx=5)

        # self.no_plume_button = Button(self.control_frame, text="No Plume", command=lambda: self.label_scene(0))
        # self.no_plume_button.pack(side="left", padx=5)

        # Save labels and scenes button
        self.save_labels_button = Button(self.control_frame, text="Save Plumes and Labels", command=self.save_plumes_and_labels)
        self.save_labels_button.pack(side="right", padx=5)

        # Plume count label
        self.plume_count_label = Label(self.control_frame, text="Total Plumes: 0")
        self.plume_count_label.pack(side="right", padx=5)

        # Frame for the Matplotlib plot
        self.plot_frame = Frame(self.root)
        self.plot_frame.pack(side="bottom", fill=BOTH, expand=True)

        # Bind arrow keys to navigation functions and 'p' key for labeling as plume
        root.bind("<Left>", lambda event: self.show_previous_scene())
        root.bind("<Right>", lambda event: self.show_next_scene())
        root.bind("<p>", lambda event: self.label_scene(1))  # 'p' key for Plume

    def load_scenes(self):
        """Load scenes based on selected year and location."""
        year = self.selected_year.get()
        location = self.selected_location.get()
        directory = f"data/test/{location}.npy"
        #directory = f"data/{year}/{location}.npy"
        #directory = f"/home/sapphire/Desktop/tropomi_scenes/{year}/s5p_l2_ch4_0017_{location}.nc_scenes.npy"
        self.scenes = np.load(directory, allow_pickle=True)
        self.current_scene_index = 0  # Reset to the first scene
        self.labels = []  # Initialize labels array
        self.plume_scenes = []  # Initialize plume scenes list
        self.plume_count = 0  # Reset plume count
        self.plume_count_label.config(text="Total Plumes: 0")  # Reset plume count display
        self.plot()

    def plot(self):
        """Plot the current scene using Matplotlib and display it in the GUI."""
        if hasattr(self, 'canvas'):
            self.canvas.get_tk_widget().destroy()

        if len(self.scenes) > 0:
            fig, ax = plt.subplots(1, 3, figsize=(15, 10), dpi=75)

            # Remove padding around the plot
            plt.subplots_adjust(left=0.03, right=0.98, top=1, bottom=0, wspace=0.1, hspace=0)
            
            # Get the current scene's data
            scene = self.scenes[self.current_scene_index]

            lat_center = scene[self.channel_map['latitude_center']][0]
            lon_center = scene[self.channel_map['longitude_center']][0]
            #view_value = scene[self.channel_map[self.channel_view.get()]][0]

            xch4_corrected = scene[self.channel_map['xch4_corrected']][0]
            normalized_ch4 = scene[self.channel_map['normalized_ch4']][0]

            min_lat = np.nanmin(lat_center) 
            max_lat = np.nanmax(lat_center) 
            min_lon = np.nanmin(lon_center) 
            max_lon = np.nanmax(lon_center)

            lat_corners = scene[self.channel_map['latitude_corners']]
            lon_corners = scene[self.channel_map['longitude_corners']]

            # Transpose lat/lon corners for plotting
            lats = np.transpose(lat_corners, (1, 2, 0))
            lons = np.transpose(lon_corners, (1, 2, 0))

            # Set color normalization and colormap
            vmin = np.nanmin(xch4_corrected)
            vmax = np.nanmax(xch4_corrected)
            cmap = plt.get_cmap('rainbow')
            norm = mcolors.Normalize(vmin=vmin, vmax=vmax)

            # Calculate new bounding box
            new_bbox = calculate_new_bbox(min_lon, max_lon, min_lat, max_lat, padding=0.2)

            # Extract u10 and v10 from the scene
            u10 = scene[self.channel_map['u10']][0]
            v10 = scene[self.channel_map['v10']][0]

            # Interpolate to fill NaNs in u10
            x_valid, y_valid = np.where(~np.isnan(u10))
            z_valid = u10[x_valid, y_valid]
            grid_x, grid_y = np.mgrid[0:u10.shape[0], 0:u10.shape[1]]
            u10_interpolated = scipy.interpolate.griddata((x_valid, y_valid), z_valid, (grid_x, grid_y), method='cubic')
            u10_interpolated = np.nan_to_num(u10_interpolated, nan=np.nanmin(u10))

            # Interpolate to fill NaNs in v10
            x_valid_v, y_valid_v = np.where(~np.isnan(v10))
            z_valid_v = v10[x_valid_v, y_valid_v]
            v10_interpolated = scipy.interpolate.griddata((x_valid_v, y_valid_v), z_valid_v, (grid_x, grid_y), method='cubic')
            v10_interpolated = np.nan_to_num(v10_interpolated, nan=np.nanmin(v10))

            # Create 12x12 mesh grid by dividing the data
            num_bins = 13
            lat_bins = np.linspace(min_lat-1, max_lat+1, num_bins + 1)
            lon_bins = np.linspace(min_lon-1, max_lon+1, num_bins + 1)

            # Initialize arrays to hold the averaged values
            u_avg = np.zeros((num_bins, num_bins))
            v_avg = np.zeros((num_bins, num_bins))
            lat_avg = np.zeros((num_bins, num_bins))
            lon_avg = np.zeros((num_bins, num_bins))

            # Iterate through each bin and calculate the mean of the data in that bin
            for i in range(num_bins):
                for j in range(num_bins):
                    lat_mask = (lat_center >= lat_bins[i]) & (lat_center < lat_bins[i + 1])
                    lon_mask = (lon_center >= lon_bins[j]) & (lon_center < lon_bins[j + 1])
                    mask = lat_mask & lon_mask
                    
                    if np.any(mask):
                        u_avg[i, j] = np.mean(u10_interpolated[mask])
                        v_avg[i, j] = np.mean(v10_interpolated[mask])
                        lat_avg[i, j] = np.mean(lat_center[mask])
                        lon_avg[i, j] = np.mean(lon_center[mask])


            # Calculate magnitudes and normalize vectors
            magnitude = np.sqrt(u_avg**2 + v_avg**2)

            # Avoid division by zero
            magnitude[magnitude == 0] = np.nan  # Set zero magnitudes to NaN to avoid division by zero

            u_avg_normalized = np.divide(u_avg, magnitude, where=~np.isnan(magnitude))
            v_avg_normalized = np.divide(v_avg, magnitude, where=~np.isnan(magnitude))

            # Replace NaNs with zeros
            u_avg_normalized = np.nan_to_num(u_avg_normalized)
            v_avg_normalized = np.nan_to_num(v_avg_normalized)

            # Optional: Scale the normalized vectors by a constant factor if needed
            scaling_factor = 0.7
            u_avg_normalized *= scaling_factor
            v_avg_normalized *= scaling_factor

            # Use the center of each bin for plotting the vectors
            lat_bin_centers = (lat_bins[:-1] + lat_bins[1:]) / 2
            lon_bin_centers = (lon_bins[:-1] + lon_bins[1:]) / 2

            lon_avg_grid, lat_avg_grid = np.meshgrid(lon_bin_centers, lat_bin_centers)

            u_avg_normalized[u_avg_normalized==0] = np.mean(u_avg_normalized)
            v_avg_normalized[v_avg_normalized==0] = np.mean(v_avg_normalized)


            u_avg[u_avg==0] = np.nan
            v_avg[v_avg==0] = np.nan


            # Create a Basemap instance
            m = Basemap(projection='cyl', ax=ax[0], llcrnrlat= new_bbox[2], urcrnrlat= new_bbox[3], llcrnrlon= new_bbox[0], urcrnrlon= new_bbox[1], suppress_ticks=False, resolution='c')
            m.bluemarble()

            
            # Add polygons for each grid cell
            for j in range(lats.shape[0]):
                for k in range(lats.shape[1]):
                    poly_corners = [
                        (lons[j, k, 0], lats[j, k, 0]),  # Top-left
                        (lons[j, k, 1], lats[j, k, 1]),  # Top-right
                        (lons[j, k, 2], lats[j, k, 2]),  # Bottom-right
                        (lons[j, k, 3], lats[j, k, 3])   # Bottom-left
                    ]
                    color = cmap(norm(xch4_corrected.flatten()[j * 32 + k]))
                    poly = Polygon(poly_corners, facecolor=color, edgecolor='grey', linewidth=0.1, alpha=0.9)
                    ax[0].add_patch(poly)

            # Plot the vectors using quiver
            m.streamplot(lon_avg_grid, lat_avg_grid, u_avg, v_avg,latlon=True,color = 'k', density=0.4, linewidth=0.7, broken_streamlines=False)

            m.quiver(lon_avg, lat_avg, u_avg, v_avg, latlon=True, scale=10, scale_units='inches', color='black', alpha=0.7, width=0.004, headwidth=3.6, headlength=5, headaxislength=4, pivot='mid', minshaft=2, minlength=1)
            ax[0].set_title('XCH4 Corrected')
                        
            # Plot the data (customize as needed)
            ax[1].imshow(xch4_corrected, cmap='rainbow')
            ax[1].set_title('XCH4 Corrected')
            ax[1].invert_yaxis()
            
            test1 = np.var(normalized_ch4[normalized_ch4 != 0])*100
            test2 = (np.max(normalized_ch4) - np.mean(normalized_ch4[normalized_ch4!=0]))*100
            # Plot the data (customize as needed)
            ax[2].imshow(normalized_ch4, cmap='rainbow')
            ax[2].set_title(f'{test1}\n{test2}')
            ax[2].invert_yaxis()

            self.canvas = FigureCanvasTkAgg(fig, master=self.plot_frame)
            self.canvas.draw()
            self.canvas.get_tk_widget().pack(side="top", fill=BOTH, expand=True)

            # Close the figure to free up memory
            plt.close(fig)

            # Update the scene label
            self.scene_label.config(text=f"Scene {self.current_scene_index + 1} of {self.scenes.shape[0]}")
        else:
            self.scene_label.config(text="No scenes available.")

    def show_previous_scene(self):
        """Show the previous scene."""
        if self.current_scene_index > 0:
            self.current_scene_index -= 1
            self.plot()

    def show_next_scene(self):
        """Show the next scene."""
        if self.current_scene_index < self.scenes.shape[0] - 1:
            self.current_scene_index += 1
            self.plot()


    def label_scene(self, label):
        """Label the current scene as 1 (Plume) or 0 (No Plume)."""
        if label == 1:
            self.plume_scenes.append(self.scenes[self.current_scene_index])  # Store only the scene if it's labeled as a plume
            self.plume_count += 1
            self.plume_count_label.config(text=f"Total Plumes: {self.plume_count}")  # Update plume count display

        self.labels.append(label)
        #print(f"Labeled scene {self.current_scene_index + 1} as {'Plume' if label == 1 else 'No Plume'}")

        # Automatically move to the next scene after labeling
        if self.current_scene_index < len(self.scenes) - 1:
            self.show_next_scene()
        else:
            pass#print("All scenes labeled.")

    def save_plumes_and_labels(self):
        """Save the plume scenes and corresponding labels to separate .npy files."""


        # Save only the plume scenes
        plume_scenes_save_path = f"data_modded/{self.selected_location.get()}_no_plume_scenes.npy"
        np.save(plume_scenes_save_path, np.array(self.plume_scenes, dtype=object))  # Save plume scenes
        #print(f"Plume scenes saved as '{plume_scenes_save_path}'.")

# Main application
if __name__ == "__main__":
    root = Tk()

    #main_directory = '/home/sapphire/Desktop/tropomi_scenes/'
    main_directory = f"data/"
    years = ['2017', '2018', '2019', '2020']
    locations = ['test']

    for sub_directories in os.listdir(main_directory):
        #for sub_sub_directories in os.listdir(main_directory + sub_directories):
            #locations.append(sub_sub_directories.split('_')[4].split('.')[0])    
        pass

    #locations = sorted(locations)

    app = SceneViewerApp(root, years, locations, channel_map)
    root.mainloop()


## Extras

### Scene Counter and Stacker

In [ ]:
import numpy as np
import os

# Directory where your .npy files are stored
directory = "data_modded/"

# List to hold all tensors
tensors = []

# Load each .npy file, check its dtype, and convert if necessary
for filename in os.listdir(directory):
    if filename.endswith(".npy"):
        tensor = np.load(os.path.join(directory, filename), allow_pickle=True)
        print(f"{filename} shape {tensor.shape}")
        
        # Check if the tensor is not already a float type
        if not np.issubdtype(tensor.dtype, np.floating):
            tensor = tensor.astype(np.float64)  # Convert to float32 if not already a float type
            #print(f"Converted {filename} to dtype {tensor.dtype}")
        tensors.append(tensor)
        #tensors.append(tensor)
        # percent = np.round(sum(tensors)/2242, 2)
        # remaining = 2242 - sum(tensors)
# print(f'{int(percent*100)} % .... {sum(tensors)} of 2242. Remaining: {remaining}')


# # # Ensure the output directory exists
output_directory = "data_modded/"
os.makedirs(output_directory, exist_ok=True)

# # # # Save the combined array to a new .npy file
output_filename = os.path.join(output_directory, "noPlumes_final.npy")
np.save(output_filename, np.concatenate(tensors, axis=0), allow_pickle=True)

# print(f"Combined tensor saved to {output_filename} with shape {combined_tensor.shape} and dtype {combined_tensor.dtype}")


In [ ]:
np.load('main_data/noPlumes_final.npy', allow_pickle=True).shape
# np.load('main_data/FINAL_SET_GOOD_PLUMES.npy', allow_pickle=True).shape

### Augmentation

In [ ]:
def rotate_scene(scene, angle):
    return scipy.ndimage.rotate(scene, angle, axes=(-2, -1), reshape=False)

# Function to flip the scene horizontally
def flip_scene_horizontally(scene):
    return np.flip(scene, axis=-1)

# Function to flip the scene vertically
def flip_scene_vertically(scene):
    return np.flip(scene, axis=-2)

# Function to apply rotations and flips to a tensor
def augment_tensor(tensor):
    augmented_tensors = []

    for scene in tensor:
        # Original scene
        augmented_tensors.append(scene)
        
        # Apply rotations
        for angle in [90, 180, 270]:
            rotated_scene = rotate_scene(scene, angle)
            augmented_tensors.append(rotated_scene)
        
        # Apply horizontal flip
        flipped_h = flip_scene_horizontally(scene)
        augmented_tensors.append(flipped_h)
        
        # Apply vertical flip
        flipped_v = flip_scene_vertically(scene)
        augmented_tensors.append(flipped_v)
        
    # Combine augmented scenes into a new tensor
    return np.array(augmented_tensors)

# Load your combined tensor
tensor = np.load("combined_scenes.npy")

# Augment the tensor with flips and rotations
augmented_tensor = augment_tensor(tensor)

# Save the augmented tensor
np.save("augmented_scenes.npy", augmented_tensor)

print(f"Augmented tensor shape: {augmented_tensor.shape}")


### Training Data Split

## Notes

### 
 - Noticed that taking the variance of the normalized matrix not inlcuding values that are 0, scenes with plumes tend to apear when the value for this above 1.

 - When we take the second highest value in the scene and subtract the mean of the normal ch4 matrix values above 0.50% tend to include plumes. Values below tend not to.
